In [1]:
# import sys
# sys.path.append('../')
# from waterbirds import *
from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader, get_eval_loader
import torchvision.transforms as transforms
import glob
import os
from utils import dict_to_str
from PIL import Image
from datamodules import WaterbirdsDataModule, ImageNet9DataModule, ToyDataModule
from torch import nn, optim, Tensor
import torch
torch.set_float32_matmul_precision('medium')
import matplotlib.pyplot as plt
import lightning as pl
from lightning_modules import *
from lightning.pytorch.callbacks import ModelCheckpoint
from madry_models import deit_base_patch16_224

In [2]:
# ckpt_path = None 
#f'models/{dataset_name}/{dict_to_str(model.hparams)}/lightning_logs/version_0/checkpoints/epoch=4-step=375.ckpt'
# if ckpt_path is not None:
#     model = Predictor.load_from_checkpoint(ckpt_path, model=vit_b_16(weights=ViT_B_16_Weights.DEFAULT), 
#                                            fc_dims=(768, 2), fc_layer='heads')

In [3]:
for mtype in ['deit_b_16']:
    for dataset_name in ['toy_datasets']:
        for lr in [5e-6]:
            for weight_decay in [3e-6]:
                if dataset_name == "waterbirds":
                    datamodule = WaterbirdsDataModule()
                    model = WaterbirdsPredictor(mtype=mtype, num_classes=datamodule.num_classes,
                                                lr=lr, weight_decay=weight_decay)
                    val_check_interval = 1.0
                    log_every_n_steps = 5
                    every_n_train_steps = None
                    epochs = 10
                    monitor_metric = 'val_acc'
                elif dataset_name == "imagenet9":
                    datamodule = ImageNet9DataModule()
                    model = ImageNet9Predictor(mtype=mtype, num_classes=datamodule.num_classes,
                                                lr=lr, weight_decay=weight_decay)
                    val_check_interval = 200
                    log_every_n_steps = 10
                    every_n_train_steps = 200
                    epochs = 2
                    monitor_metric = 'orig_val_acc'
                else:
                    datamodule = ToyDataModule(f'/cmlscratch/sriramb/{dataset_name}/')
                    model = ToyDatasetPredictor(mtype=mtype, num_classes=datamodule.num_classes,
                                                lr=lr, weight_decay=weight_decay)
                    val_check_interval = 1.0
                    log_every_n_steps = 5
                    every_n_train_steps = None
                    epochs = 1
                    monitor_metric = 'corr_val_acc'

                checkpoint_callback = ModelCheckpoint(
                                            save_top_k=3,
                                            save_last=True,
                                            every_n_train_steps=every_n_train_steps,
                                            monitor=monitor_metric,
                                            mode="max",
                                            dirpath=f"models/{dataset_name}/{dict_to_str(model.hparams)}",
                                            filename="{epoch:02d}-{step:04d}-{monitor:.2f}".replace('monitor',monitor_metric),
                                        )
                trainer = pl.Trainer(
                                    default_root_dir=f"models/{dataset_name}",
                                    callbacks=[checkpoint_callback],
                                    log_every_n_steps=5,
                                    val_check_interval=val_check_interval,
        #                             fast_dev_run=5,
                                    max_epochs=epochs)
                ckpt_path = f"models/{dataset_name}/{dict_to_str(model.hparams)}/last.ckpt"
                if not os.path.exists(ckpt_path):
                    ckpt_path = None
                trainer.fit(model=model, datamodule=datamodule, ckpt_path=ckpt_path)
    #         break

==>[Loaded PyTorch-pretrained deit checkpoint.]


/cmlscratch/sriramb/anaconda3/envs/relai/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /cmlscratch/sriramb/anaconda3/envs/relai/lib/python3 ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
/cmlscratch/sriramb/anaconda3/envs/relai/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /cmlscratch/sriramb/an

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
